# MTH5 Example 07

## Make an MTH5 from Phoenix Data

This example demonstrates how to read Phoenix data into an MTH5 file.  These data are provided by Phoenix Geophysical Ltd. Provided are 3 synchronously recored stations.  The Phoenix instruments MTU-5 series are broadband instruments with multiple sample rates.  

The instrumnet is constantly recording at 24 kHz and depending on the input paramters those data are decimated or chunked into different data files.  A common setup is to record continuously at 150 samples per second and record short bursts of 24 kHz for just a few seconds every 15 minutes or so.  In this case the continuous 24 kHz is decimated down to 150 Hz and stored in files with a length of 6 minutes.  The short bursts are also stored into a separate file.  This is done for each channel recorded.  Its common for broadband instruments to record each channel individually making it easier to buffer.  

A challenge arises when you have the short bursts is that you will have a ton of runs.  Luckily HDF5 can handle this, we just have to organize it properly.  And figure out which file belongs where.  A `PhoenixCollection` object is provided for just this task. 

## Imports

In [2]:
from pathlib import Path

from mth5.mth5 import MTH5
from mth5 import read_file
from mth5.io.phoenix import PhoenixCollection #  ReceiverMetadataJSON,

## Data Directory

Specify the station directory.  Phoenix files place each channel in a folder under the station directory named by the channel number.  There is also a `recmeta.json` file that has metadata output by the receiver that can be useful.  In the `PhoenixGeopPy/sample_data` there are 2 folders one for native data, these are `.bin` files which are the raw data in counts sampled at 24k.  There is also a folder for segmented files, these files are calibrated to millivolts and decimated or segmented data according to the recording configuration.  Most of the time you would use the segmented files? 

In [3]:
station_dir = Path().cwd().parent.parent.joinpath("data", "time_series", "phoenix")

## File Collection

We've developed a collection dataframe to help sort out which files are which and which files can be grouped together into runs.  Continous runs will be given a single run name and segmented data will have sequential run names.  Both will have the pattern `sr{sample_rate}_####` for the `run.id`.

Here `PhoenixCollection.get_runs` returns a two level ordered dictionary (`OrderedDict`).  The first level is keyed by station ID.  These objects are in turn ordered dictionaries by run ID.  Therefore you can loop over stations and runs.

#### Receiver Metadata

The data logger or receiver will output a `JSON` file that contains useful metadata that is missing from the data files.  The `recmeta.json` file can be read into an object with methods to translate to `mt_metadata` objects. This is read in by `PhoenixCollection` and is in the attribute `receiver_metadata`. 


In [4]:
phx_collection = PhoenixCollection(file_path=station_dir)
run_dict = phx_collection.get_runs(sample_rates=[150, 24000])

## Initiate MTH5

First initiate an MTH5 file, can use the receiver metadata to fill in some `Survey` metadata

In [5]:
m = MTH5()
m.open_mth5(station_dir.joinpath("mth5_from_phoenix.h5"), "w")

2024-10-05T15:05:25.204674-0700 | INFO | mth5.mth5 | _initialize_file | Initialized MTH5 0.2.0 file /home/kkappler/software/irismt/earthscope-mt-course/data/time_series/phoenix/mth5_from_phoenix.h5 in mode w


/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
        --> Dataset: channel_summary
        ..............................
        --> Dataset: fc_summary
        .........................
        --> Dataset: tf_summary
        .........................

## Loop through Surveys, Stations, Runs, and Channels

Using the `run_dict` value output by the `PhoenixCollection.get_runs` we can simply loop through the runs without knowing if the data are continous or discontinuous, the `read_file` will take care of that.

Users should note the Phoenix file structure.  Inside the folder are files with extensions of `.td_24k` and `td_150`.  

- `.td_24k` are usually bursts of a few seconds of data sampled at 24k samples per second to get high frequency information.  The returned object is a `mth5.timeseries.ChannelTS`.
- `td_150` is data continuously sampled at 150 samples per second.  These files usually have a set length, commonly an hour. The returned object is a `mth5.timeseries.ChannelTS`.



In [8]:
%%time

for station_id, station_dict in run_dict.items():
    
    phx_metadata = phx_collection.metadata_dict[station_id]
    
    survey_group = m.add_survey(
        phx_metadata.survey_metadata.id,
        survey_metadata=phx_metadata.survey_metadata)
    
    station_metadata = phx_metadata.station_metadata
    station_group = survey_group.stations_group.add_station(
        station_metadata.id, 
        station_metadata=station_metadata
    )
    for run_id, run_df in station_dict.items():
        run_metadata = phx_metadata.run_metadata
        run_metadata.id = run_id
        run_metadata.sample_rate = float(run_df.sample_rate.unique()[0])

        run_group = station_group.add_run(run_metadata.id, run_metadata=run_metadata)
        for row in run_df.itertuples():
            ch_ts = read_file(row.fn, **{"channel_map":phx_metadata.channel_map})
            ch_metadata = phx_metadata.get_ch_metadata(
                ch_ts.channel_metadata.channel_number
            )
            # need to update the time period and sample rate as estimated from the data not the metadata
            ch_metadata.sample_rate = ch_ts.sample_rate
            ch_metadata.time_period.update(ch_ts.channel_metadata.time_period)
            ch_ts.channel_metadata.update(ch_metadata)

            # add channel to the run group
            ch_dataset = run_group.from_channel_ts(ch_ts)
            
        # update run metadata with information from the input channels
        # run_group.validate_run_metadata()
        
    # update the station metadata based on new runs
    # station_group.validate_station_metadata()
    
    # update survey metadata based on new station
    survey_group.update_metadata()
        

2024-10-05T15:08:14.906712-0700 | INFO | mth5.groups.survey | add_survey | survey MIST already exists, returning existing group.
2024-10-05T15:08:14.911103-0700 | INFO | mth5.groups.base | _add_group | StationGroup 9042 already exists, returning existing group.
2024-10-05T15:08:14.916747-0700 | INFO | mth5.groups.base | _add_group | RunGroup sr150_0001 already exists, returning existing group.
2024-10-05T15:08:16.662750-0700 | WARNING | mt_metadata.timeseries.filters.filtered | _check_consistency | Filter consistency check failed for an unknown reason
2024-10-05T15:08:16.663395-0700 | WARNING | mt_metadata.timeseries.filters.filtered | applied | Filter names and applied lists are not the same size. Be sure to check the inputs. names = ['v_to_mv'], applied = [True, True]
2024-10-05T15:08:17.560649-0700 | WARNING | mt_metadata.timeseries.filters.filtered | _check_consistency | Filter consistency check failed for an unknown reason
2024-10-05T15:08:17.561141-0700 | WARNING | mt_metadata.ti

In [9]:
m.channel_summary.summarize()
m.channel_summary.to_dataframe()

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,has_data,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,none,9042,sr150_0001,33.094975,-115.87206,-55.429401,e1,2024-03-28 11:41:21+00:00,2024-03-29 09:07:36.993333333+00:00,11576400,150.0,electric,0.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,none,9042,sr150_0001,33.094975,-115.87206,-55.429401,e2,2024-03-28 11:41:21+00:00,2024-03-29 09:07:36.993333333+00:00,11576400,150.0,electric,90.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
2,none,9042,sr150_0001,33.094975,-115.87206,-55.429401,h1,2024-03-28 11:41:21+00:00,2024-03-29 09:07:36.993333333+00:00,11576400,150.0,magnetic,0.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
3,none,9042,sr150_0001,33.094975,-115.87206,-55.429401,h2,2024-03-28 11:41:21+00:00,2024-03-29 09:07:36.993333333+00:00,11576400,150.0,magnetic,90.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4,none,9042,sr24k_0001,33.094975,-115.87206,-55.429401,e1,2024-03-28 18:41:42+00:00,2024-03-28 18:41:42.999958333+00:00,24000,24000.0,electric,0.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,none,9042,sr24k_0214,33.094975,-115.87206,-55.429401,h2,2024-03-29 15:59:42+00:00,2024-03-29 15:59:42.999958333+00:00,24000,24000.0,magnetic,90.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
860,none,9042,sr24k_0215,33.094975,-115.87206,-55.429401,e1,2024-03-29 16:05:42+00:00,2024-03-29 16:05:42.999958333+00:00,24000,24000.0,electric,0.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
861,none,9042,sr24k_0215,33.094975,-115.87206,-55.429401,e2,2024-03-29 16:05:42+00:00,2024-03-29 16:05:42.999958333+00:00,24000,24000.0,electric,90.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
862,none,9042,sr24k_0215,33.094975,-115.87206,-55.429401,h1,2024-03-29 16:05:42+00:00,2024-03-29 16:05:42.999958333+00:00,24000,24000.0,magnetic,0.0,0.0,volts,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


In [10]:
m.close_mth5()

2024-10-05T15:10:39.239688-0700 | INFO | mth5.mth5 | close_mth5 | Flushing and closing /home/kkappler/software/irismt/earthscope-mt-course/data/time_series/phoenix/mth5_from_phoenix.h5
